# 第3部 第2章 単回帰モデル

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 2.1 本章の目的と概要
参考書籍から引用します。\
RとStanの部分をNumPyroに読み替えてください。

> **テーマ**\
> 本章ではStanを用いた単回帰モデルの推定方法を述べます。

> **目的**\
> RとStanうぃ用いた一般化線形モデル推定のごく初歩を学ぶ目的で,本章を執筆しました.\
> モデルを推定するコードの書き方だけではなく,説明変数によって応答変数の平均値が変化するモデルを解釈する方法を学んでください。

> **概要**\
> 分析の準備 → データの読み込みと可視化 → モデルの構造\
> → Stanファイルの実装 → MCMCの実行 → 事後分布の可視化

## 2.2 分析の準備
ライブラリimportの節です。\
この記事では既にimport済みだとします。

## データの読み込みと可視化
架空のビール売り上げデータと気温のデータを読み込みます。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-2-1-beer-sales-2.csv")
display(df.head())

# サンプルサイズを表示する
print( len(df) )

In [ ]:
X = jnp.array(df["temperature"], dtype = float)
Y = jnp.array(df["sales"], dtype = float)

売り上げと気温の関係を散布図で確認します。

In [ ]:
ax = sns.scatterplot(data = df, x = "temperature", y = "sales")
ax.set_title("図3.2.1 ビールの売り上げと気温の散布図")

## 2.4 モデルの構造
ビールの売り上げ $y_{i}$ と気温 $x_{i}$ との関係を数式で整理します。\
参考書籍では次のように書かれています。\
1行目の関係式はリンク関数(恒等関数)と呼ばれており,目的変数の平均値が説明変数によって変化することを想定しています。

$$
\begin{aligned}
\mu_{i} =& \beta_{0} + \beta_{1} x_{i}\\
y_{i} \sim& Normal(\mu_{i}, \sigma^2)
\end{aligned}
$$

この結論への道筋を示します。\
まず、ビールの売り上げ $y_{i}$ と気温 $x_{i}$ を1次関数近似できると仮定します。

$$
y_{i} \approx \beta_{0} + \beta_{1} x_{i}
$$

右辺を $\mu_{i}$ とおきます。

$$
\begin{aligned}
\mu_{i} \equiv& \beta_{0} + \beta_{1} x_{i}\\
y_{i} \approx& \mu_{i}
\end{aligned}
$$

次に、ビールの売り上げ $y_{i}$ が気温 $x_{i}$ によって決まる値 $\mu_{i}$ を期待値とする正規分布に従うと仮定します。

$$
\begin{aligned}
y_{i} \sim& Normal(\mu_{i}, \sigma^2)\\
\mu_{i} \equiv& \beta_{0} + \beta_{1} x_{i}
\end{aligned}
$$

NumPyroで確率モデルを実装するときは最後の式を下から記述していきます。

## 2.5 単回帰モデルのためのStanﾌｧｲﾙの実装
NumPyroのモデルを作成します。\
NumPyroはStanと異なり,パラメータの事前分布を明示する必要があります。

In [ ]:
def model_linear_regression(X, Y = None):
    '''
        第3部 2章 のビールの売り上げ
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X)
    with numpyro.plate("N", N):
        X = numpyro.deterministic("X", X)
    # 1次関数の切片と傾きの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 20))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 20))
    # リンク関数(恒等関数)を定義する
    μ = numpyro.deterministic("μ", β0 + β1 * X)
    # 目的変数が従う正規分布の標準偏差の事前分布を設定する
    σ = numpyro.sample("σ", dist.HalfNormal(scale = 20))
    # 目的変数 y は説明変数 x の値に応じた平均値 μ をパラメータとする正規分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Normal(loc = μ, scale = σ), obs = Y)

作成したモデルをプロットします。

In [ ]:
model_args = {
    "X": X,
    "Y": Y
}
try_render_model(model_linear_regression, render_name = "ビールの売り上げと気温の線形回帰", **model_args)

## 2.6 MCMCの実行
データを用意してモデルを作成したら後はユーティリティ関数に渡すだけです。

In [ ]:
model_args = {
    "X": X,
    "Y": Y
}
idata = run_mcmc(
    model_linear_regression,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    log_likelihood = False,
    **model_args
)

## 2.7 事後分布の可視化
サンプリングが上手くいったか確認します。\
書籍と同じ結果を得られました。

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["β0", "β1", "σ"])
plt.tight_layout()

## 終わりに